# Air quality project

## Project and data description

### Data

The dataset consists air quality data (the concentrations of particulate matter PM1, PM2.5 and PM10, temperature, air pressure and humidity) from 2017 generated by network of 56 low-cost sensors located in Krakow, Poland. 

The data is divided into 12 files- one for each month. Each row represent single represents a single measurement. Each column shows measured amount:

- date nad time in UTC (Coordinated Universal Time) 

As Krakow, Poland is located in UTC+1 (in winter) and UTC+2 (in summer) final forecas should take under consideration this difference.  

- temperature

Air temperature measured in Celcius degrees- standard temprerature unit for Poland

- humidity

Current air humidity presented in percentages 

- pressure

Current air pressure presented in Pascals. May require conversion to hectopascals as it is a common unit for presenting air pressure value for forecast


Next, we can find three columns with measured particular matter in 3 cattegories- each in separate column:  

- pm10

Current value of particulate matter with size less than 10 μm (micrometre). Presented in μg/m3 (microgram on cubic metres). 

According to norms set by Chief Inspectorate of Environmental Protection (Główny Inspektorat Ochrony Środowiska- GIOŚ) the daily average concentrations of PM10 should not exceed 50 micrograms per cubic meter. 

- pm25

Amount of particulate matter with size less than 2.5 μm. Shows concentracion level in μg/m3. 

The amout is given in  Simirally as particular matter with size less than 1μm, it can also penetrate from lungs to blood. 

According to norms set by GIOŚ the daily average concentrations of PM2.5 should not exceed 25 micrograms per cubic meter. 

- pm1

Measured level of particulate matter with size less than 1 μm. Presented in μg/m3 (microgram on cubic metres). 

The allowable concentration for pm1 has not been defined.


The source of particulate matter in air are various kinds of combustion ex. domestic boiler rooms or even barbecues. As research of Chief Inspectorate of Environmental Protection (Główny Inspektorat Ochrony Środowiska- GIOŚ) shows, main source of this kind of air poluttion are low quality domestic heating stoves. Particular matter can easily penetrate from lungs to bloodstream. The smaller the particle, the more dangerous it is, as it can enter the bloodstream more easily. 

Each column name (except 'UTC time') has numeric prefix. The prefix is sensor id. For example in column named '140_humidity' we can find air humidity mesured by sensor number 140. 

There are 56 sensors, so as a result we get 337 columns (56 times 6 features + one UTC time column). 

### The goal

The goal of a project is to build a model for forecasting the amount of particulate matter pm1, pm 2.5 and pm 10, based on prior mesurements. 

## Preprocessing and analysis

### Imports

In [1]:
import glob
import os
import numpy as np
import pandas as pd
import folium
import random
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

### Uploading and transforming the data

The data are in separated 12 files- each for one month. We're uploading all files to one dataframe. 

In [2]:
def create_data_frame(path_to_files, file_name_end):
    all_data = []
    all_files = glob.glob(os.path.join(path_to_files,file_name_end))
    
    for path in all_files:
        frame = pd.read_csv(path)
        all_data.append(frame)
    
    df = pd.concat(all_data, axis=0, ignore_index=True)
    
    return df

In [3]:
root_path = r'C:\Users\User\PycharmProjects\Air_quality\Data_files'
file_name_like = '*2017.csv'

df = create_data_frame(root_path, file_name_like)

In [4]:
df

,UTC time,3_temperature,3_humidity,3_pressure,3_pm1,3_pm25,3_pm10,140_temperature,140_humidity,140_pressure,...,857_pressure,857_pm1,857_pm25,857_pm10,895_temperature,895_humidity,895_pressure,895_pm1,895_pm25,895_pm10
0,2017-04-01T00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,6.0,92.0,101906.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-04-01T01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,6.0,92.0,101869.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-04-01T02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,5.0,94.0,101837.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-04-01T03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,5.0,92.0,101834.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-04-01T04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,4.0,94.0,101832.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8588,2017-09-30T19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,9.0,83.0,102373.0,...,102337.0,35.0,35.0,72.0,8.0,83.0,102383.0,37.0,36.0,67.0
8589,2017-09-30T20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,8.0,83.0,102374.0,...,102341.0,36.0,37.0,76.0,8.0,83.0,102391.0,37.0,37.0,69.0
8590,2017-09-30T21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,7.0,90.0,102380.0,...,102349.0,38.0,39.0,79.0,6.0,90.0,102402.0,45.0,45.0,79.0
8591,2017-09-30T22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,7.0,88.0,102380.0,...,102355.0,44.0,44.0,87.0,6.0,88.0,102408.0,46.0,46.0,79.0


We're sorting the data frame by 'UTC time' column.

In [5]:
def sort_by_column(frame, column_name):
    sorted_frame = frame.sort_values(by=column_name)
    sorded_frame = sorted_frame.reset_index(drop=True)
    
    return sorted_frame

In [6]:
df = sort_by_column(df, 'UTC time')
df

,UTC time,3_temperature,3_humidity,3_pressure,3_pm1,3_pm25,3_pm10,140_temperature,140_humidity,140_pressure,...,857_pressure,857_pm1,857_pm25,857_pm10,895_temperature,895_humidity,895_pressure,895_pm1,895_pm25,895_pm10
2713,2017-01-01T00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2714,2017-01-01T01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2715,2017-01-01T02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2716,2017-01-01T03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2717,2017-01-01T04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2036,2017-12-24T20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,6.0,87.0,101736.0,...,101726.0,12.0,15.0,31.0,5.0,85.0,101798.0,13.0,15.0,26.0
2037,2017-12-24T21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,6.0,88.0,101776.0,...,101771.0,12.0,14.0,29.0,5.0,85.0,101840.0,12.0,13.0,23.0
2038,2017-12-24T22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,6.0,85.0,101785.0,...,101788.0,10.0,12.0,26.0,5.0,82.0,101839.0,10.0,12.0,21.0
2039,2017-12-24T23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,5.0,88.0,101782.0,...,101774.0,9.0,11.0,24.0,5.0,85.0,101837.0,10.0,12.0,21.0


### Sensor location

To better understand the data we're creating a map with sensor locations. 

First we're uploading data frame with sensors locations (from csv file). 

In [7]:
def upload_sensor_frame(file_path):
    
    sensor_loc_path = file_path
    sensor_loc_df = pd.read_csv(sensor_loc_path)
    #sensor_loc_df.set_index('id', drop=True)
    
    return sensor_loc_df

In [8]:
sensor_loc_path = r'C:\Users\User\PycharmProjects\Air_quality\Data_files\sensor_locations.csv'
location_frame = upload_sensor_frame(sensor_loc_path)

Now we're creating a map with markers pointing sensor location. After pointing the marker with the cursor, we can see the sensor id.  

In [9]:
def create_a_map(start_point, zoom):

    sensor_loc_map = folium.Map(
                                location=start_point,
                                zoom_start= zoom
                              )
    return sensor_loc_map

In [10]:
location=[50.057748,19.961375]
initial_zoom=12

empty_map = create_a_map(location, initial_zoom)

In [11]:
def add_markers(location_df, my_map):
    for _, sensor in location_df.iterrows():
        
        icon_color='blue'
        
        folium.Marker(location = [sensor['latitude'], sensor['longitude']],
                      #popup=sensor['id'],
                      tooltip=sensor['id'],
                      icon=folium.Icon(color=icon_color)
                     ).add_to(my_map)

    return my_map
        
sensor_map = add_markers(location_frame, empty_map)
sensor_map

### Data preparation

Firstly we reset the index. Next we convert UTC time values to datetime. 

In [13]:
df = df.reset_index(drop=True)

In [14]:
df['UTC time'] = pd.to_datetime(df['UTC time'])

In [ ]:
Now basic peek into the data frame. 

In [16]:
df.head()

,UTC time,3_temperature,3_humidity,3_pressure,3_pm1,3_pm25,3_pm10,140_temperature,140_humidity,140_pressure,...,857_pressure,857_pm1,857_pm25,857_pm10,895_temperature,895_humidity,895_pressure,895_pm1,895_pm25,895_pm10
0,2017-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8593 entries, 0 to 8592
Columns: 337 entries, UTC time to 895_pm10
dtypes: datetime64[ns](1), float64(336)
memory usage: 22.1 MB


In [146]:
df.describe()

,3_temperature,3_humidity,3_pressure,3_pm1,3_pm25,3_pm10,140_temperature,140_humidity,140_pressure,140_pm1,...,857_pressure,857_pm1,857_pm25,857_pm10,895_temperature,895_humidity,895_pressure,895_pm1,895_pm25,895_pm10
count,206.000000,206.000000,1178.000000,1235.000000,1235.000000,1235.000000,6506.000000,6506.000000,6683.000000,6631.000000,...,2653.000000,2653.000000,2653.000000,2653.000000,5443.000000,5443.000000,6337.000000,6337.000000,6337.000000,6337.000000
mean,-5.131068,74.718447,101652.880306,70.830769,75.454251,112.540891,11.464494,80.271288,101639.084842,26.537023,...,101591.979269,29.469280,31.978892,65.852997,10.660665,79.759691,101660.566672,23.879754,23.922519,41.092788
std,2.174645,9.939247,1020.704947,54.853035,66.299222,89.265573,7.630920,17.167440,653.937345,19.283222,...,809.559197,21.697085,27.709362,56.194746,7.864320,19.397843,649.831280,16.803287,20.897541,32.908029
min,-9.000000,38.000000,99160.000000,2.000000,0.000000,3.000000,-6.000000,21.000000,98374.000000,0.000000,...,98372.000000,0.000000,0.000000,0.000000,-7.000000,18.000000,98433.000000,0.000000,0.000000,0.000000
25%,-7.000000,69.000000,100828.500000,33.000000,30.000000,52.000000,6.000000,70.000000,101291.000000,13.000000,...,101116.000000,15.000000,14.000000,27.000000,4.000000,69.000000,101328.000000,13.000000,11.000000,18.000000
50%,-6.000000,77.000000,101726.000000,50.000000,52.000000,87.000000,11.000000,86.000000,101666.000000,23.000000,...,101614.000000,25.000000,25.000000,53.000000,10.000000,85.000000,101686.000000,21.000000,19.000000,33.000000
75%,-4.000000,82.000000,102352.000000,101.000000,107.500000,153.000000,17.000000,93.000000,102064.000000,34.000000,...,102207.000000,37.000000,41.000000,85.000000,16.000000,92.000000,102079.000000,29.000000,30.000000,54.000000
max,2.000000,92.000000,103619.000000,249.000000,314.000000,440.000000,37.000000,115.000000,103155.000000,160.000000,...,103146.000000,151.000000,190.000000,373.000000,34.000000,134.000000,103199.000000,132.000000,159.000000,239.000000


Having data from all sensors in one data frame makes it difficult to analyze. We decide to spread data from each sensor to separate data frame. 

There has not beed provided separate frame or column with sensor ids, so we're extracting them from column names. 

In [17]:
def get_sensor_ids(data_frame):
    sensor_ids=[]
    for column in data_frame.columns:
        id = column.partition('_')[0]
        if id.isdigit():
            id = int(id)
            sensor_ids.append(id)
    ids = set(sensor_ids)
    ids = list(ids)
    # ids.sort()
    return ids

We're saving them in 'sensor_ids' list. 

In [18]:
sensor_ids = get_sensor_ids(df)
sensor_ids

Alter that we create a dictionary with data frames for every sensor. Sensor id i a dictionary key.

In [20]:
def create_sensor_frame(all_sensors_frame, sensor_id):
    sensor_id = str(sensor_id)
    sensor_frame = all_sensors_frame.loc[:,all_sensors_frame.columns.str.startswith(sensor_id)]
    id_series = pd.Series(int(sensor_id), name='sensor_id', index=range(len(all_sensors_frame['UTC time'])))
    sensor_frame = pd.concat([id_series, all_sensors_frame['UTC time'], sensor_frame], axis=1)
    return sensor_frame

,sensor_id,UTC time,142_temperature,142_humidity,142_pressure,142_pm1,142_pm25,142_pm10
0,142,2017-01-01 00:00:00,-4.0,68.0,102279.0,115.0,127.0,196.0
1,142,2017-01-01 01:00:00,-5.0,68.0,102228.0,105.0,116.0,180.0
2,142,2017-01-01 02:00:00,-5.0,66.0,102149.0,110.0,121.0,189.0
3,142,2017-01-01 03:00:00,-5.0,66.0,102097.0,107.0,118.0,184.0
4,142,2017-01-01 04:00:00,-5.0,65.0,102047.0,112.0,123.0,193.0
...,...,...,...,...,...,...,...,...
8588,142,2017-12-24 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN
8589,142,2017-12-24 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN
8590,142,2017-12-24 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN
8591,142,2017-12-24 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
def all_sensor_dict(all_data_frame, sensor_list):
    frame_dictionary = {}
    for sensor_id in sensor_list:
        frame_dictionary[sensor_id] = create_sensor_frame(all_data_frame, sensor_id)
    return frame_dictionary

In [23]:
sensor_dict = all_sensor_dict(df, sensor_ids)

## Data analysis

### Repleacing nans.

In [28]:
for sensor_id in sensor_dict:
    sensor_dict[sensor_id].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8593 entries, 0 to 8592
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   sensor_id      8593 non-null   int64         
 1   UTC time       8593 non-null   datetime64[ns]
 2   3_temperature  206 non-null    float64       
 3   3_humidity     206 non-null    float64       
 4   3_pressure     1178 non-null   float64       
 5   3_pm1          1235 non-null   float64       
 6   3_pm25         1235 non-null   float64       
 7   3_pm10         1235 non-null   float64       
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 537.2 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8593 entries, 0 to 8592
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   sensor_id        8593 non-null   int64         
 1   UTC time         8593 non-null   datetime64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8593 entries, 0 to 8592
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   sensor_id        8593 non-null   int64         
 1   UTC time         8593 non-null   datetime64[ns]
 2   211_temperature  1449 non-null   float64       
 3   211_humidity     1448 non-null   float64       
 4   211_pressure     1424 non-null   float64       
 5   211_pm1          1449 non-null   float64       
 6   211_pm25         1449 non-null   float64       
 7   211_pm10         1449 non-null   float64       
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 537.2 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8593 entries, 0 to 8592
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   sensor_id        8593 non-null   int64         
 1   UTC time         8593 n

It seems that many sensors has missing values. We start by identifying the ratio of nans in each column. 

In [29]:
def nans_ratio(data_frame):
    print('Liczba brakujących wartości w poszczególnych kolumnach:')
    column_names = data_frame.columns
    for column in column_names:
        print(column,': ', ((data_frame[column].isnull().sum())/len(data_frame[column])).round(2))

In [30]:
for sensor_id in sensor_dict:
    nans_ratio(sensor_dict[sensor_id])

Liczba brakujących wartości w poszczególnych kolumnach:
sensor_id :  0.0
UTC time :  0.0
3_temperature :  0.98
3_humidity :  0.98
3_pressure :  0.86
3_pm1 :  0.86
3_pm25 :  0.86
3_pm10 :  0.86
Liczba brakujących wartości w poszczególnych kolumnach:
sensor_id :  0.0
UTC time :  0.0
263_temperature :  0.02
263_humidity :  0.02
263_pressure :  0.02
263_pm1 :  0.02
263_pm25 :  0.02
263_pm10 :  0.02
Liczba brakujących wartości w poszczególnych kolumnach:
sensor_id :  0.0
UTC time :  0.0
140_temperature :  0.24
140_humidity :  0.24
140_pressure :  0.22
140_pm1 :  0.23
140_pm25 :  0.23
140_pm10 :  0.23
Liczba brakujących wartości w poszczególnych kolumnach:
sensor_id :  0.0
UTC time :  0.0
142_temperature :  0.76
142_humidity :  0.76
142_pressure :  0.76
142_pm1 :  0.77
142_pm25 :  0.77
142_pm10 :  0.77
Liczba brakujących wartości w poszczególnych kolumnach:
sensor_id :  0.0
UTC time :  0.0
147_temperature :  0.73
147_humidity :  0.73
147_pressure :  0.08
147_pm1 :  0.07
147_pm25 :  0.07
147_

We can see that ratios can vary from nearly 0 to nearly 100. To handle such diversity in the amount of nans, we decide to split the process into few steps. 

At first we select the columns where missing data exceed 45%. 

In [31]:
def high_nans_ratio(data_frame):
    column_names = data_frame.columns
    for column in column_names:
        nan_ratio = ((data_frame[column].isnull().sum())/len(data_frame[column])).round(2)
        if nan_ratio > 0.45:
            print(column,': ', nan_ratio)

In [32]:
def apply_function(dict_of_frames, function):
    for sensor_id in dict_of_frames:
        function(dict_of_frames[sensor_id])

In [33]:
apply_function(sensor_dict, high_nans_ratio)

3_temperature :  0.98
3_humidity :  0.98
3_pressure :  0.86
3_pm1 :  0.86
3_pm25 :  0.86
3_pm10 :  0.86
142_temperature :  0.76
142_humidity :  0.76
142_pressure :  0.76
142_pm1 :  0.77
142_pm25 :  0.77
142_pm10 :  0.77
147_temperature :  0.73
147_humidity :  0.73
808_temperature :  0.87
808_humidity :  0.87
808_pressure :  0.87
808_pm1 :  0.87
808_pm25 :  0.87
808_pm10 :  0.87
172_temperature :  0.81
172_humidity :  0.81
176_pressure :  0.8
178_temperature :  0.77
178_humidity :  0.77
178_pressure :  1.0
178_pm1 :  0.77
178_pm25 :  0.77
178_pm10 :  0.77
179_pressure :  0.82
180_pressure :  0.8
181_pressure :  0.8
182_pressure :  0.8
183_pressure :  0.89
184_pressure :  0.8
185_pressure :  0.83
187_temperature :  0.72
187_humidity :  0.72
187_pressure :  0.72
187_pm1 :  0.72
187_pm25 :  0.72
187_pm10 :  0.72
196_pressure :  0.8
201_pressure :  0.8
203_pressure :  1.0
205_temperature :  0.71
205_humidity :  0.71
205_pressure :  0.84
205_pm1 :  0.71
205_pm25 :  0.71
205_pm10 :  0.71
713_

We can see that, there are sensors where there are more than 45% lack of data in every column. Completing such a large amount of data could cause problems in model training, so we decided to exclude these sensors from further analysis. 

In [35]:
def high_nans_removal(data_frame_dict):
    dropped_frames_miss_6 = {}
    new_data_frame_dict = {}
    for dict_key in sensor_dict:
    #for k, frame in enumerate(data_frame_dict):
        n_missing_cols = (sensor_dict[dict_key].isna().sum() / len(sensor_dict[dict_key]) > 0.45).sum()
        
        if n_missing_cols == 6:
            dropped_frames_miss_6[dict_key]=sensor_dict[dict_key]
        else:
            #new_data_frame_list.append(frame)
            new_data_frame_dict[dict_key]=sensor_dict[dict_key]
            
    return new_data_frame_dict, dropped_frames_miss_6

In [38]:
new_sensor_dict, dropped_frames = high_nans_removal(sensor_dict)

After excluding sensors with insuficient amount of data, we check if there are any columns with lack of data exceeding 45% left. 

In [40]:
apply_function(new_sensor_dict, high_nans_ratio)

147_temperature :  0.73
147_humidity :  0.73
172_temperature :  0.81
172_humidity :  0.81
176_pressure :  0.8
179_pressure :  0.82
180_pressure :  0.8
181_pressure :  0.8
182_pressure :  0.8
183_pressure :  0.89
184_pressure :  0.8
185_pressure :  0.83
196_pressure :  0.8
201_pressure :  0.8
203_pressure :  1.0


There are still columns with high nan ratios, however there are columns for temperature, humidity and pressure. This features do not change drasticly in small geographical areas, so we decide to complete the data with measurements from other sensor. We choose sensor to take data from based on geographical proximity and neighbourhood similarity. We also need to avoid using the same sensor twice, or extracting data from excluded sensors. 

In [44]:
new_sensor_dict[172]['172_temperature'] = new_sensor_dict[170]['170_temperature']

In [45]:
new_sensor_dict[172]['172_humidity'] = new_sensor_dict[170]['170_humidity']

In [46]:
new_sensor_dict[196]['196_pressure'].isna().sum()

6863

In [47]:
new_sensor_dict[196]['196_pressure'] = new_sensor_dict[214]['214_pressure']

In [48]:
new_sensor_dict[196]['196_pressure'].isna().sum()

67

In [49]:
new_sensor_dict[180]['180_pressure'].isna().sum()

6855

In [50]:
new_sensor_dict[180]['180_pressure'] = new_sensor_dict[223]['223_pressure']

In [51]:
new_sensor_dict[180]['180_pressure'].isna().sum()

116

In [52]:
new_sensor_dict[201]['201_pressure'].isna().sum()

6854

In [53]:
new_sensor_dict[201]['201_pressure'] = new_sensor_dict[196]['196_pressure']

In [54]:
new_sensor_dict[201]['201_pressure'].isna().sum()

67

In [55]:
new_sensor_dict[179]['179_pressure'].isna().sum()

7020

In [56]:
new_sensor_dict[179]['179_pressure'] = new_sensor_dict[220]['220_pressure']

In [57]:
new_sensor_dict[179]['179_pressure'].isna().sum()

57

In [58]:
new_sensor_dict[147]['147_temperature'].isna().sum()

6292

In [59]:
new_sensor_dict[147]['147_humidity'].isna().sum()

6294

In [60]:
new_sensor_dict[147]['147_temperature'] = new_sensor_dict[189]['189_temperature']
new_sensor_dict[147]['147_humidity'] = new_sensor_dict[189]['189_humidity']

In [61]:
new_sensor_dict[147]['147_temperature'].isna().sum(), new_sensor_dict[147]['147_humidity'].isna().sum()

(131, 133)

In [62]:
new_sensor_dict[184]['184_pressure'].isna().sum()

6860

In [63]:
new_sensor_dict[184]['184_pressure'] = new_sensor_dict[173]['173_pressure']

In [64]:
new_sensor_dict[184]['184_pressure'].isna().sum()

119

In [65]:
new_sensor_dict[182]['182_pressure'].isna().sum()

6892

In [66]:
new_sensor_dict[182]['182_pressure'] = new_sensor_dict[180]['180_pressure']

In [67]:
new_sensor_dict[182]['182_pressure'].isna().sum()

116

In [68]:
new_sensor_dict[176]['176_pressure'].isna().sum()

6854

In [69]:
new_sensor_dict[176]['176_pressure'] = new_sensor_dict[147]['147_pressure']

In [70]:
new_sensor_dict[176]['176_pressure'].isna().sum()

653

In [71]:
new_sensor_dict[183]['183_pressure'].isna().sum()

7639

In [72]:
new_sensor_dict[183]['183_pressure'] = new_sensor_dict[209]['209_pressure']

In [73]:
new_sensor_dict[183]['183_pressure'].isna().sum()

792

In [74]:
new_sensor_dict[181]['181_pressure'].isna().sum()

6859

In [75]:
new_sensor_dict[181]['181_pressure'] = new_sensor_dict[204]['204_pressure']

In [76]:
new_sensor_dict[181]['181_pressure'].isna().sum()

77

In [77]:
new_sensor_dict[203]['203_pressure'].isna().sum()

8593

In [78]:
new_sensor_dict[203]['203_pressure'] = new_sensor_dict[194]['194_pressure']

In [79]:
new_sensor_dict[203]['203_pressure'].isna().sum()

288

In [80]:
new_sensor_dict[185]['185_pressure'].isna().sum()

7135

In [81]:
new_sensor_dict[185]['185_pressure'] = new_sensor_dict[192]['192_pressure']

In [82]:
new_sensor_dict[185]['185_pressure'].isna().sum()

234

In [83]:
# def nans_ratio(data_frame):
#     column_names = data_frame.columns
#     for column in column_names:
#         nan_ratio = ((data_frame[column].isnull().sum())/len(data_frame[column])).round(2)
#         print(column,': ', nan_ratio)

In [84]:
apply_function(new_sensor_dict, nans_ratio)

sensor_id :  0.0
UTC time :  0.0
263_temperature :  0.02
263_humidity :  0.02
263_pressure :  0.02
263_pm1 :  0.02
263_pm25 :  0.02
263_pm10 :  0.02
sensor_id :  0.0
UTC time :  0.0
140_temperature :  0.24
140_humidity :  0.24
140_pressure :  0.22
140_pm1 :  0.23
140_pm25 :  0.23
140_pm10 :  0.23
sensor_id :  0.0
UTC time :  0.0
147_temperature :  0.02
147_humidity :  0.02
147_pressure :  0.08
147_pm1 :  0.07
147_pm25 :  0.07
147_pm10 :  0.07
sensor_id :  0.0
UTC time :  0.0
169_temperature :  0.04
169_humidity :  0.04
169_pressure :  0.04
169_pm1 :  0.04
169_pm25 :  0.04
169_pm10 :  0.04
sensor_id :  0.0
UTC time :  0.0
170_temperature :  0.02
170_humidity :  0.02
170_pressure :  0.02
170_pm1 :  0.02
170_pm25 :  0.02
170_pm10 :  0.02
sensor_id :  0.0
UTC time :  0.0
171_temperature :  0.01
171_humidity :  0.01
171_pressure :  0.01
171_pm1 :  0.01
171_pm25 :  0.01
171_pm10 :  0.01
sensor_id :  0.0
UTC time :  0.0
172_temperature :  0.02
172_humidity :  0.02
172_pressure :  0.03
172_pm1

In [85]:
def nans_ratio_above_10proc(data_frame):
    #print('Liczba brakujących wartości w poszczególnych kolumnach:')
    column_names = data_frame.columns
    for column in column_names:
        nan_ratio = ((data_frame[column].isnull().sum())/len(data_frame[column])).round(2)
        if nan_ratio > 0.1:
            print(column,': ', nan_ratio)

In [86]:
apply_function(new_sensor_dict, nans_ratio_above_10proc)

140_temperature :  0.24
140_humidity :  0.24
140_pressure :  0.22
140_pm1 :  0.23
140_pm25 :  0.23
140_pm10 :  0.23
174_temperature :  0.14
174_humidity :  0.14
174_pressure :  0.14
174_pm1 :  0.14
174_pm25 :  0.14
174_pm10 :  0.14
177_pressure :  0.35
182_temperature :  0.12
182_humidity :  0.12
182_pm1 :  0.12
182_pm25 :  0.12
182_pm10 :  0.12
195_temperature :  0.15
195_humidity :  0.15
195_pressure :  0.16
195_pm1 :  0.15
195_pm25 :  0.15
195_pm10 :  0.15
203_temperature :  0.18
203_humidity :  0.18
203_pm1 :  0.18
203_pm25 :  0.18
203_pm10 :  0.18
622_temperature :  0.38
622_humidity :  0.38
622_pressure :  0.35
622_pm1 :  0.35
622_pm25 :  0.35
622_pm10 :  0.35
895_temperature :  0.37
895_humidity :  0.37
895_pressure :  0.26
895_pm1 :  0.26
895_pm25 :  0.26
895_pm10 :  0.26


In [87]:
def nans_ratio_below_2proc(data_frame):
    #print('Liczba brakujących wartości w poszczególnych kolumnach:')
    column_names = data_frame.columns
    for column in column_names:
        nan_ratio = ((data_frame[column].isnull().sum())/len(data_frame[column])).round(2)
        if nan_ratio < 0.015:
            print(column,': ', nan_ratio)

In [88]:
apply_function(new_sensor_dict, nans_ratio_below_2proc)

sensor_id :  0.0
UTC time :  0.0
sensor_id :  0.0
UTC time :  0.0
sensor_id :  0.0
UTC time :  0.0
sensor_id :  0.0
UTC time :  0.0
sensor_id :  0.0
UTC time :  0.0
sensor_id :  0.0
UTC time :  0.0
171_temperature :  0.01
171_humidity :  0.01
171_pressure :  0.01
171_pm1 :  0.01
171_pm25 :  0.01
171_pm10 :  0.01
sensor_id :  0.0
UTC time :  0.0
sensor_id :  0.0
UTC time :  0.0
173_pressure :  0.01
173_pm1 :  0.01
173_pm25 :  0.01
173_pm10 :  0.01
sensor_id :  0.0
UTC time :  0.0
sensor_id :  0.0
UTC time :  0.0
176_temperature :  0.01
176_humidity :  0.01
176_pm1 :  0.01
176_pm25 :  0.01
176_pm10 :  0.01
sensor_id :  0.0
UTC time :  0.0
sensor_id :  0.0
UTC time :  0.0
179_pressure :  0.01
sensor_id :  0.0
UTC time :  0.0
180_temperature :  0.0
180_humidity :  0.0
180_pressure :  0.01
180_pm1 :  0.0
180_pm25 :  0.0
180_pm10 :  0.0
sensor_id :  0.0
UTC time :  0.0
181_pressure :  0.01
sensor_id :  0.0
UTC time :  0.0
182_pressure :  0.01
sensor_id :  0.0
UTC time :  0.0
sensor_id :  0.0

In [89]:
def nan_ratio_for_frame(sensor_dictionary):
    nan_ratio_sums = {}
    for key in sensor_dictionary:
        column_names = sensor_dictionary[key].columns
        nan_ratio_sum = 0
        for column in column_names:
            nan_ratio= ((sensor_dictionary[key][column].isnull().sum())/(len(sensor_dictionary[key][column])))
            nan_ratio_sum += nan_ratio
        nan_ratio_sums[key] = nan_ratio_sum
    return nan_ratio_sums

In [90]:
nan_results = nan_ratio_for_frame(new_sensor_dict)

In [91]:
def ten_min_values(ratio_sums_dict):
    top_10_values = sorted(ratio_sums_dict.values())[:10]
    final_score = []
    for value in top_10_values:
        for key in ratio_sums_dict:
            if ratio_sums_dict[key] == value:
                final_score.append(key)
    return set(final_score) 

In [92]:
min_nan_ratio_sensors = ten_min_values(nan_results)
min_nan_ratio_sensors

{171, 180, 204, 212, 214, 218, 220, 221, 226, 228}

In [93]:
new_sensor_dict[212]

,sensor_id,UTC time,212_temperature,212_humidity,212_pressure,212_pm1,212_pm25,212_pm10
0,212,2017-01-01 00:00:00,0.0,29.0,101435.0,107.0,123.0,196.0
1,212,2017-01-01 01:00:00,0.0,28.0,101387.0,123.0,144.0,227.0
2,212,2017-01-01 02:00:00,0.0,28.0,101302.0,120.0,140.0,224.0
3,212,2017-01-01 03:00:00,0.0,27.0,101250.0,104.0,119.0,190.0
4,212,2017-01-01 04:00:00,0.0,25.0,101204.0,88.0,100.0,161.0
...,...,...,...,...,...,...,...,...
8588,212,2017-12-24 20:00:00,6.0,77.0,101624.0,10.0,11.0,19.0
8589,212,2017-12-24 21:00:00,5.0,77.0,101659.0,9.0,10.0,18.0
8590,212,2017-12-24 22:00:00,5.0,74.0,101658.0,7.0,8.0,15.0
8591,212,2017-12-24 23:00:00,5.0,77.0,101649.0,7.0,8.0,14.0


In [94]:
column_name = str(212) + '_pressure'
column_name

'212_pressure'

In [95]:
def pascals_to_hectopascals(sensors_dictionary):
    for key in sensors_dictionary:
        column_name = str(key) + '_pressure'
        new_sensor_dict[key][column_name] = new_sensor_dict[key][column_name]/100

In [96]:
pascals_to_hectopascals(new_sensor_dict)

In [97]:
new_sensor_dict[212]

,sensor_id,UTC time,212_temperature,212_humidity,212_pressure,212_pm1,212_pm25,212_pm10
0,212,2017-01-01 00:00:00,0.0,29.0,1014.35,107.0,123.0,196.0
1,212,2017-01-01 01:00:00,0.0,28.0,1013.87,123.0,144.0,227.0
2,212,2017-01-01 02:00:00,0.0,28.0,1013.02,120.0,140.0,224.0
3,212,2017-01-01 03:00:00,0.0,27.0,1012.50,104.0,119.0,190.0
4,212,2017-01-01 04:00:00,0.0,25.0,1012.04,88.0,100.0,161.0
...,...,...,...,...,...,...,...,...
8588,212,2017-12-24 20:00:00,6.0,77.0,1016.24,10.0,11.0,19.0
8589,212,2017-12-24 21:00:00,5.0,77.0,1016.59,9.0,10.0,18.0
8590,212,2017-12-24 22:00:00,5.0,74.0,1016.58,7.0,8.0,15.0
8591,212,2017-12-24 23:00:00,5.0,77.0,1016.49,7.0,8.0,14.0


In [98]:
new_sensor_dict[212].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8593 entries, 0 to 8592
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   sensor_id        8593 non-null   int64         
 1   UTC time         8593 non-null   datetime64[ns]
 2   212_temperature  8585 non-null   float64       
 3   212_humidity     8584 non-null   float64       
 4   212_pressure     8560 non-null   float64       
 5   212_pm1          8585 non-null   float64       
 6   212_pm25         8585 non-null   float64       
 7   212_pm10         8585 non-null   float64       
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 537.2 KB


In [99]:
new_sensor_dict[212].loc[546:548,:]

,sensor_id,UTC time,212_temperature,212_humidity,212_pressure,212_pm1,212_pm25,212_pm10
546,212,2017-01-23 18:00:00,-3.0,34.0,1013.02,119.0,138.0,218.0
547,212,2017-01-23 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN
548,212,2017-01-23 20:00:00,-4.0,30.0,1013.64,114.0,134.0,213.0


In [100]:
# shows all rows with nan in any column
new_sensor_dict[212][new_sensor_dict[212].isna().any(axis=1)]

,sensor_id,UTC time,212_temperature,212_humidity,212_pressure,212_pm1,212_pm25,212_pm10
547,212,2017-01-23 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN
1189,212,2017-02-19 13:00:00,4.0,81.0,NaN,19.0,15.0,27.0
1190,212,2017-02-19 14:00:00,3.0,79.0,NaN,27.0,25.0,43.0
1191,212,2017-02-19 15:00:00,3.0,78.0,NaN,32.0,31.0,53.0
1192,212,2017-02-19 16:00:00,3.0,76.0,NaN,32.0,32.0,53.0
1193,212,2017-02-19 17:00:00,1.0,76.0,NaN,31.0,30.0,51.0
1194,212,2017-02-19 18:00:00,1.0,76.0,NaN,32.0,31.0,53.0
1195,212,2017-02-19 19:00:00,0.0,75.0,NaN,38.0,39.0,66.0
1196,212,2017-02-19 20:00:00,0.0,75.0,NaN,49.0,51.0,82.0
1197,212,2017-02-19 21:00:00,0.0,74.0,NaN,63.0,68.0,99.0


In [101]:
# temperature = new_sensor_dict[212]['212_temperature']


# for index, value in enumerate(temperature):
#     print(type(value))
#     if np.isnan(value) and index !=0:
#         prev_val = temperature[index-1]
#         print(prev_val)
#         temperature[index]=prev_val

In [102]:
def replace_nan_with_prev(data_frame):
    for column in data_frame:
        if column[0:3].isdigit():
            for index, value in enumerate(data_frame[column]):
                float(value)
                if np.isnan([value]) and index !=0:
                    prev_val = data_frame[column][index-1]
                    data_frame[column][index]=prev_val

In [103]:
# sensors chosen to model
min_nan_ratio_sensors

{171, 180, 204, 212, 214, 218, 220, 221, 226, 228}

In [104]:
replace_nan_with_prev(new_sensor_dict[212])

C:\Users\User\AppData\Local\Temp\ipykernel_11372\1483693759.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame[column][index]=prev_val


In [105]:
new_sensor_dict[212][new_sensor_dict[212].isna().any(axis=1)]

,sensor_id,UTC time,212_temperature,212_humidity,212_pressure,212_pm1,212_pm25,212_pm10


In [106]:
new_sensor_dict[226][new_sensor_dict[226].isna().any(axis=1)]

,sensor_id,UTC time,226_temperature,226_humidity,226_pressure,226_pm1,226_pm25,226_pm10
1189,226,2017-02-19 13:00:00,3.0,82.0,NaN,21.0,17.0,30.0
1190,226,2017-02-19 14:00:00,3.0,81.0,NaN,28.0,26.0,43.0
1191,226,2017-02-19 15:00:00,3.0,81.0,NaN,29.0,28.0,46.0
1192,226,2017-02-19 16:00:00,2.0,78.0,NaN,19.0,15.0,26.0
1193,226,2017-02-19 17:00:00,1.0,78.0,NaN,23.0,20.0,33.0
1194,226,2017-02-19 18:00:00,1.0,79.0,NaN,35.0,34.0,58.0
1195,226,2017-02-19 19:00:00,0.0,79.0,NaN,43.0,43.0,71.0
1196,226,2017-02-19 20:00:00,0.0,78.0,NaN,54.0,55.0,88.0
1197,226,2017-02-19 21:00:00,0.0,77.0,NaN,54.0,56.0,89.0
1198,226,2017-02-19 22:00:00,0.0,78.0,NaN,56.0,58.0,91.0


In [107]:
replace_nan_with_prev(new_sensor_dict[226])

C:\Users\User\AppData\Local\Temp\ipykernel_11372\1483693759.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame[column][index]=prev_val


In [108]:
new_sensor_dict[226][new_sensor_dict[226].isna().any(axis=1)]

,sensor_id,UTC time,226_temperature,226_humidity,226_pressure,226_pm1,226_pm25,226_pm10


In [109]:
new_sensor_dict[220][new_sensor_dict[220].isna().any(axis=1)]

,sensor_id,UTC time,220_temperature,220_humidity,220_pressure,220_pm1,220_pm25,220_pm10
547,220,2017-01-23 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN
1189,220,2017-02-19 13:00:00,4.0,79.0,NaN,23.0,20.0,34.0
1190,220,2017-02-19 14:00:00,4.0,78.0,NaN,36.0,35.0,58.0
1191,220,2017-02-19 15:00:00,3.0,79.0,NaN,34.0,33.0,56.0
1192,220,2017-02-19 16:00:00,2.0,76.0,NaN,31.0,30.0,51.0
1193,220,2017-02-19 17:00:00,1.0,76.0,NaN,33.0,32.0,54.0
1194,220,2017-02-19 18:00:00,1.0,76.0,NaN,36.0,36.0,60.0
1195,220,2017-02-19 19:00:00,0.0,76.0,NaN,47.0,47.0,79.0
1196,220,2017-02-19 20:00:00,0.0,75.0,NaN,51.0,52.0,86.0
1197,220,2017-02-19 21:00:00,0.0,75.0,NaN,66.0,67.0,100.0


**dopytać o tę uwagę**

In [110]:
replace_nan_with_prev(new_sensor_dict[220])

C:\Users\User\AppData\Local\Temp\ipykernel_11372\1483693759.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame[column][index]=prev_val


In [111]:
new_sensor_dict[220][new_sensor_dict[220].isna().any(axis=1)]

,sensor_id,UTC time,220_temperature,220_humidity,220_pressure,220_pm1,220_pm25,220_pm10


In [112]:
set(min_nan_ratio_sensors)

{171, 180, 204, 212, 214, 218, 220, 221, 226, 228}

In [113]:
pd.set_option('display.max_rows',200)

In [114]:
new_sensor_dict[171][new_sensor_dict[171].isna().any(axis=1)]

,sensor_id,UTC time,171_temperature,171_humidity,171_pressure,171_pm1,171_pm25,171_pm10
547,171,2017-01-23 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN
1189,171,2017-02-19 13:00:00,4.0,81.0,NaN,25.0,22.0,37.0
1190,171,2017-02-19 14:00:00,4.0,79.0,NaN,28.0,26.0,46.0
1191,171,2017-02-19 15:00:00,4.0,78.0,NaN,34.0,33.0,56.0
1192,171,2017-02-19 16:00:00,3.0,76.0,NaN,32.0,30.0,52.0
1193,171,2017-02-19 17:00:00,1.0,76.0,NaN,30.0,29.0,48.0
1194,171,2017-02-19 18:00:00,1.0,76.0,NaN,37.0,36.0,61.0
1195,171,2017-02-19 19:00:00,1.0,76.0,NaN,46.0,46.0,76.0
1196,171,2017-02-19 20:00:00,0.0,74.0,NaN,54.0,55.0,86.0
1197,171,2017-02-19 21:00:00,0.0,75.0,NaN,68.0,71.0,103.0


In [115]:
replace_nan_with_prev(new_sensor_dict[171])

C:\Users\User\AppData\Local\Temp\ipykernel_11372\1483693759.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame[column][index]=prev_val


In [116]:
new_sensor_dict[171][new_sensor_dict[171].isna().any(axis=1)]

,sensor_id,UTC time,171_temperature,171_humidity,171_pressure,171_pm1,171_pm25,171_pm10


In [117]:
new_sensor_dict[180][new_sensor_dict[180].isna().any(axis=1)]

,sensor_id,UTC time,180_temperature,180_humidity,180_pressure,180_pm1,180_pm25,180_pm10
371,180,2017-01-16 11:00:00,NaN,NaN,1019.48,35.0,34.0,63.0
428,180,2017-01-18 20:00:00,NaN,NaN,1038.60,NaN,NaN,NaN
429,180,2017-01-18 21:00:00,NaN,NaN,1038.53,NaN,NaN,NaN
430,180,2017-01-18 22:00:00,NaN,NaN,1038.34,NaN,NaN,NaN
431,180,2017-01-18 23:00:00,NaN,NaN,1038.01,NaN,NaN,NaN
432,180,2017-01-19 00:00:00,NaN,NaN,1037.90,NaN,NaN,NaN
433,180,2017-01-19 01:00:00,NaN,NaN,1037.99,NaN,NaN,NaN
434,180,2017-01-19 02:00:00,NaN,NaN,1038.01,NaN,NaN,NaN
435,180,2017-01-19 03:00:00,NaN,NaN,1037.93,NaN,NaN,NaN
436,180,2017-01-19 04:00:00,NaN,NaN,1037.49,NaN,NaN,NaN


In [118]:
replace_nan_with_prev(new_sensor_dict[180])

C:\Users\User\AppData\Local\Temp\ipykernel_11372\1483693759.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame[column][index]=prev_val


In [119]:
new_sensor_dict[171][new_sensor_dict[171].isna().any(axis=1)]

,sensor_id,UTC time,171_temperature,171_humidity,171_pressure,171_pm1,171_pm25,171_pm10


In [120]:
set(min_nan_ratio_sensors)

{171, 180, 204, 212, 214, 218, 220, 221, 226, 228}

In [121]:
new_sensor_dict[204][new_sensor_dict[204].isna().any(axis=1)]

,sensor_id,UTC time,204_temperature,204_humidity,204_pressure,204_pm1,204_pm25,204_pm10
547,204,2017-01-23 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN
1189,204,2017-02-19 13:00:00,4.0,90.0,NaN,29.0,28.0,47.0
1190,204,2017-02-19 14:00:00,4.0,89.0,NaN,33.0,33.0,54.0
1191,204,2017-02-19 15:00:00,4.0,90.0,NaN,34.0,34.0,56.0
1192,204,2017-02-19 16:00:00,3.0,88.0,NaN,30.0,29.0,49.0
1193,204,2017-02-19 17:00:00,2.0,88.0,NaN,31.0,31.0,51.0
1194,204,2017-02-19 18:00:00,2.0,88.0,NaN,40.0,41.0,67.0
1195,204,2017-02-19 19:00:00,1.0,87.0,NaN,53.0,57.0,86.0
1196,204,2017-02-19 20:00:00,0.0,86.0,NaN,75.0,82.0,116.0
1197,204,2017-02-19 21:00:00,0.0,86.0,NaN,86.0,95.0,134.0


In [122]:
replace_nan_with_prev(new_sensor_dict[204])

C:\Users\User\AppData\Local\Temp\ipykernel_11372\1483693759.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame[column][index]=prev_val


In [123]:
new_sensor_dict[204][new_sensor_dict[204].isna().any(axis=1)]

,sensor_id,UTC time,204_temperature,204_humidity,204_pressure,204_pm1,204_pm25,204_pm10


In [124]:
new_sensor_dict[214][new_sensor_dict[214].isna().any(axis=1)]

,sensor_id,UTC time,214_temperature,214_humidity,214_pressure,214_pm1,214_pm25,214_pm10
1189,214,2017-02-19 13:00:00,3.0,87.0,NaN,24.0,21.0,36.0
1190,214,2017-02-19 14:00:00,3.0,85.0,NaN,27.0,25.0,42.0
1191,214,2017-02-19 15:00:00,3.0,85.0,NaN,34.0,33.0,56.0
1192,214,2017-02-19 16:00:00,2.0,83.0,NaN,31.0,29.0,48.0
1193,214,2017-02-19 17:00:00,0.0,83.0,NaN,30.0,28.0,46.0
1194,214,2017-02-19 18:00:00,0.0,84.0,NaN,34.0,34.0,57.0
1195,214,2017-02-19 19:00:00,0.0,84.0,NaN,39.0,40.0,67.0
1196,214,2017-02-19 20:00:00,0.0,85.0,NaN,51.0,53.0,87.0
1197,214,2017-02-19 21:00:00,0.0,85.0,NaN,72.0,74.0,109.0
1198,214,2017-02-19 22:00:00,0.0,85.0,NaN,59.0,61.0,97.0


In [125]:
replace_nan_with_prev(new_sensor_dict[214])

C:\Users\User\AppData\Local\Temp\ipykernel_11372\1483693759.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame[column][index]=prev_val


In [126]:
new_sensor_dict[214][new_sensor_dict[214].isna().any(axis=1)]

,sensor_id,UTC time,214_temperature,214_humidity,214_pressure,214_pm1,214_pm25,214_pm10


In [127]:
new_sensor_dict[218][new_sensor_dict[218].isna().any(axis=1)]

,sensor_id,UTC time,218_temperature,218_humidity,218_pressure,218_pm1,218_pm25,218_pm10
1189,218,2017-02-19 13:00:00,4.0,79.0,NaN,42.0,44.0,68.0
1190,218,2017-02-19 14:00:00,3.0,78.0,NaN,58.0,68.0,104.0
1191,218,2017-02-19 15:00:00,4.0,78.0,NaN,39.0,41.0,69.0
1192,218,2017-02-19 16:00:00,3.0,76.0,NaN,64.0,73.0,107.0
1193,218,2017-02-19 17:00:00,1.0,75.0,NaN,62.0,68.0,98.0
1194,218,2017-02-19 18:00:00,1.0,76.0,NaN,60.0,65.0,94.0
1195,218,2017-02-19 19:00:00,0.0,75.0,NaN,71.0,79.0,112.0
1196,218,2017-02-19 20:00:00,0.0,74.0,NaN,76.0,85.0,120.0
1197,218,2017-02-19 21:00:00,0.0,73.0,NaN,87.0,98.0,137.0
1198,218,2017-02-19 22:00:00,0.0,74.0,NaN,82.0,91.0,128.0


In [128]:
replace_nan_with_prev(new_sensor_dict[218])

C:\Users\User\AppData\Local\Temp\ipykernel_11372\1483693759.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame[column][index]=prev_val


In [129]:
new_sensor_dict[218][new_sensor_dict[218].isna().any(axis=1)]

,sensor_id,UTC time,218_temperature,218_humidity,218_pressure,218_pm1,218_pm25,218_pm10


In [130]:
set(min_nan_ratio_sensors)

{171, 180, 204, 212, 214, 218, 220, 221, 226, 228}

In [131]:
new_sensor_dict[221][new_sensor_dict[221].isna().any(axis=1)]

,sensor_id,UTC time,221_temperature,221_humidity,221_pressure,221_pm1,221_pm25,221_pm10
1189,221,2017-02-19 13:00:00,3.0,81.0,NaN,22.0,19.0,33.0
1190,221,2017-02-19 14:00:00,3.0,79.0,NaN,25.0,23.0,40.0
1191,221,2017-02-19 15:00:00,3.0,78.0,NaN,30.0,29.0,51.0
1192,221,2017-02-19 16:00:00,3.0,76.0,NaN,30.0,28.0,49.0
1193,221,2017-02-19 17:00:00,1.0,76.0,NaN,25.0,23.0,38.0
1194,221,2017-02-19 18:00:00,1.0,77.0,NaN,31.0,30.0,52.0
1195,221,2017-02-19 19:00:00,1.0,77.0,NaN,37.0,37.0,63.0
1196,221,2017-02-19 20:00:00,0.0,75.0,NaN,47.0,48.0,77.0
1197,221,2017-02-19 21:00:00,0.0,76.0,NaN,61.0,65.0,93.0
1198,221,2017-02-19 22:00:00,0.0,77.0,NaN,58.0,61.0,90.0


In [132]:
replace_nan_with_prev(new_sensor_dict[221])

C:\Users\User\AppData\Local\Temp\ipykernel_11372\1483693759.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame[column][index]=prev_val


In [133]:
new_sensor_dict[221][new_sensor_dict[221].isna().any(axis=1)]

,sensor_id,UTC time,221_temperature,221_humidity,221_pressure,221_pm1,221_pm25,221_pm10


In [134]:
new_sensor_dict[226][new_sensor_dict[226].isna().any(axis=1)]

,sensor_id,UTC time,226_temperature,226_humidity,226_pressure,226_pm1,226_pm25,226_pm10


In [135]:
new_sensor_dict[228][new_sensor_dict[228].isna().any(axis=1)]

,sensor_id,UTC time,228_temperature,228_humidity,228_pressure,228_pm1,228_pm25,228_pm10
1189,228,2017-02-19 13:00:00,4.0,80.0,NaN,30.0,30.0,48.0
1190,228,2017-02-19 14:00:00,4.0,79.0,NaN,30.0,28.0,49.0
1191,228,2017-02-19 15:00:00,4.0,79.0,NaN,34.0,34.0,58.0
1192,228,2017-02-19 16:00:00,3.0,76.0,NaN,24.0,20.0,35.0
1193,228,2017-02-19 17:00:00,1.0,76.0,NaN,26.0,23.0,39.0
1194,228,2017-02-19 18:00:00,1.0,77.0,NaN,43.0,43.0,73.0
1195,228,2017-02-19 19:00:00,0.0,76.0,NaN,50.0,51.0,83.0
1196,228,2017-02-19 20:00:00,0.0,76.0,NaN,74.0,80.0,116.0
1197,228,2017-02-19 21:00:00,0.0,75.0,NaN,59.0,60.0,95.0
1198,228,2017-02-19 22:00:00,0.0,76.0,NaN,57.0,60.0,97.0


In [136]:
replace_nan_with_prev(new_sensor_dict[228])

C:\Users\User\AppData\Local\Temp\ipykernel_11372\1483693759.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame[column][index]=prev_val


In [137]:
new_sensor_dict[228][new_sensor_dict[228].isna().any(axis=1)]

,sensor_id,UTC time,228_temperature,228_humidity,228_pressure,228_pm1,228_pm25,228_pm10


In [138]:
pd.set_option('display.max_rows',10)

In [139]:
new_sensor_dict[171]

,sensor_id,UTC time,171_temperature,171_humidity,171_pressure,171_pm1,171_pm25,171_pm10
0,171,2017-01-01 00:00:00,1.0,27.0,1024.37,147.0,170.0,256.0
1,171,2017-01-01 01:00:00,0.0,27.0,1023.87,129.0,150.0,228.0
2,171,2017-01-01 02:00:00,1.0,27.0,1023.10,134.0,157.0,240.0
3,171,2017-01-01 03:00:00,0.0,27.0,1022.60,144.0,171.0,261.0
4,171,2017-01-01 04:00:00,0.0,27.0,1022.12,137.0,162.0,248.0
...,...,...,...,...,...,...,...,...
8588,171,2017-12-24 20:00:00,6.0,88.0,1016.91,13.0,14.0,24.0
8589,171,2017-12-24 21:00:00,6.0,88.0,1017.26,12.0,13.0,21.0
8590,171,2017-12-24 22:00:00,5.0,85.0,1017.38,10.0,11.0,19.0
8591,171,2017-12-24 23:00:00,5.0,88.0,1017.44,10.0,11.0,19.0


## Training/ testing sets

In [140]:
def create_train_set(sensor_dictionary, sensor_ids, n_train_samples): #(sensor_dictionary,sensor_ids, n_samples):
    train_sensor_ids= sensor_ids[:n_train_samples]
    test_sensor_ids= sensor_ids[n_train_samples:]
    train_set = {}
    test_set = {}
    for frame in sensor_dictionary:
        if frame in train_sensor_ids:
            train_set[frame] = sensor_dictionary[frame]
    for frame in sensor_dictionary:
        if frame in test_sensor_ids:
            test_set[frame] = sensor_dictionary[frame]
    return train_set, test_set

In [141]:
min_nan_ratio_sensors = list(set(min_nan_ratio_sensors))
min_nan_ratio_sensors

[226, 228, 171, 204, 212, 180, 214, 218, 220, 221]

In [142]:
train_set, test_set = create_train_set(new_sensor_dict, min_nan_ratio_sensors, 8)
train_set.keys()

dict_keys([171, 180, 204, 212, 214, 218, 226, 228])

In [143]:
test_set.keys()

dict_keys([220, 221])

In [144]:
create_train_set(min_nan_ratio_sensors, 8)

TypeError: create_train_set() missing 1 required positional argument: 'n_train_samples'

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
plt.locator_params(axis='x', nbins=20)
ax.plot(df['UTC time'], df['220_temperature'])
ax.xaxis.set_tick_params(rotation=90)
#plt.plot(df['UTC time'], df['3_temperature'])
plt.show()

# może być temp. średnia, albo dla dnia (też średnia)

## Standarization

In [ ]:
for column in train_set[171]:
    if column[0:3].isdigit():
        column_mean = train_set[171][column].mean()
        column_std = train_set[171][column].std()
        train_set[171][column] = (train_set[171][column] - column_mean)/column_std

In [ ]:
# scaler = StandardScaler()

# scaled_train_set = scaler.fit_transform(train_set[171])

#train_set[171].describe()
def standarize_frame(sensor_set):
standarized_set = {}

for sensor_frame in sensor_set:
    for column in train_set[171]:
        if column[0:3].isdigit():
            column_mean = train_set[171][column].mean()
            column_std = train_set[171][column].std()
            train_set[171][column] = (train_set[171][column] - column_mean)/column_std

In [ ]:
train_set[171]

In [ ]:
train_set[171].describe()

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
plt.locator_params(axis='x', nbins=20)
ax.plot(new_sensor_dict[172]['UTC time'], new_sensor_dict[172]['172_temperature'])
ax.xaxis.set_tick_params(rotation=90)
plt.show()

Czujnik 857 uruchomiony we wrześniu. Dalej ciągłość zapisów. Co w tej sytuacji? 

### Odległości między punktami i wysokość npm 

odległości między punktami- gotowe biblioteki. 

import geopy.distance

coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)

print geopy.distance.geodesic(coords_1, coords_2).km

import mpu

Point one
lat1 = 52.2296756
lon1 = 21.0122287

Point two
lat2 = 52.406374
lon2 = 16.9251681

What you were looking for
dist = mpu.haversine_distance((lat1, lon1), (lat2, lon2))
print(dist)  # gives 278.45817507541943.

https://wrf-python.readthedocs.io/en/latest/internal_api/generated/wrf.g_geoht.get_height.html

In [ ]:
tutaj spróbować:
https://gis.stackexchange.com/questions/338392/getting-elevation-for-multiple-lat-long-coordinates-in-python

import rasterio

coords = ((147.363,-36.419), (147.361,-36.430))
elevation = 'srtm_66_20.tif'

with rasterio.open(elevation) as src:
    vals = src.sample(coords)
    for val in vals:
        print(val[0]) #val is an array of values, 1 element 
                      #per band. src is a single band raster 
                      #so we only need val[0]

Do tworzenia wysokości: 

import requests
import urllib
import pandas as pd
 
USGS Elevation Point Query Service
url = r'https://nationalmap.gov/epqs/pqs.php?'
 
coordinates with known elevation 
lat = [48.633, 48.733, 45.1947, 45.1962]
lon = [-93.9667, -94.6167, -93.3257, -93.2755]
 
create data frame
df = pd.DataFrame({
    'lat': lat,
    'lon': lon
})
 
def elevation_function(df, lat_column, lon_column):
    """Query service using lat, lon. add the elevation values as a new column."""
    elevations = []
    for lat, lon in zip(df[lat_column], df[lon_column]):
 
        # define rest query params
        params = {
            'output': 'json',
            'x': lon,
            'y': lat,
            'units': 'Meters'
        }
 
        # format query string and return query value
        result = requests.get((url + urllib.parse.urlencode(params)))
        elevations.append(result.json()['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation'])
 
    df['elev_meters'] = elevations

`elevation_function(df, 'lat', 'lon')`